In [7]:
import sys
import numpy as np
import re
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import os
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
from os import listdir
from os.path import isfile, join
from matplotlib import cm
from matplotlib.cm import ScalarMappable
import pandas as pd
import glob
from scipy import interpolate
print(sys.executable)
print(sys.path)
# sys.path.insert(0,'C:\\Users\\Villads\\anaconda3\\pkgs')
import sklearn
from sklearn.decomposition import PCA
matplotlib.rcParams.update({'font.size': 18})  
from matplotlib.animation import FuncAnimation

#%% Define some functions to be used

def read_data(file_name, num_columns=2, sep=','):
    i = 0
    data = [[] for _ in range(num_columns)]
    with open(os.path.join(file_name), mode='r') as csvfile:
        for row in csvfile:  
            if i > 0:
                row = row.split(sep)
                for j in range(num_columns):
                    data[j].append(float(row[j]))
            i += 1
    return np.array(data)
def line_function(x1,y1,x2,y2, x):
    # Calculate the slope and intercept of the line
    slope = (y2 - y1) / (x2 - x1)
    intercept = y1 - slope * x1
    # Calculate the y-values of the line for the given x-values
    y = slope * x + intercept
    return y

def read_powerfile(file_name, num_columns=2, sep=','):
    i = 0
    data = [[] for _ in range(num_columns)]
    with open(os.path.join(file_name), mode='r') as csvfile:
        for row in csvfile:  
            if i > 0 and i%2 ==0:
                row = row.split(sep)
                for j in range(num_columns):
                    data[j].append(float(row[j]))
            i += 1
    return np.array(data)

def extract_number(filename):
    # Extract the numeric part after "__ExcWL" using regular expression
    match = re.search(r'_(\d+)(?=__ExcWL)', filename)
    return int(match.group(1)) if match else float('inf')

def read_folder_ofSpectra_andPowerCorrect(path, num_columns=2, sep=','):
    files = [os.path.join(path, f) for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and "_PowerCalibrationFile_" not in f]
    #Sorting
    files = sorted(files, key=extract_number)
    all_data = []
    power_corr = read_powerfile(os.path.join(path, '_PowerCalibrationFile_.csv'), num_columns=2, sep=',')
    for file_name,ycorr in zip(files,power_corr[1]):
        print(file_name)
        if not file_name == '_PowerCalibrationFile_':
            data = read_data(file_name, num_columns, sep)
  #          plt.plot(data[0],data[num_columns-1]/ycorr)
            y_corr = y_corr_func(data[0])
            if correct:     
                all_data.append(data[num_columns-1]/ycorr / y_corr)
            if not correct:
                all_data.append(data[num_columns-1]/ycorr) 
  #  plt.title('data_loaded')
  #  plt.show()
    return data[0],np.array(all_data)

def read_file(path, num_columns=2, sep=',', correct=True):
    files = [os.path.join(path, f) for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    # Sorting
    files = sorted(files, key=extract_number)
    all_data = []
    for file_name in files:
        print(file_name)
        data = read_data(file_name, num_columns, sep)
        all_data.append(data[num_columns-1])
    return data[0], np.array(all_data)
    
def smoothing(x,data):
    y = data  + max(data)
   # plt.plot(x,y)
    percentage_diff = np.abs(np.diff(y))
    rolling_avg = np.convolve(y, np.ones(5)/5, mode='valid',)
    diff_from_avg = (y[4:] - rolling_avg) /rolling_avg
    outliers_mask = diff_from_avg > 0.1
    # Replace outliers with the average of neighboring points
    outliers = np.where(outliers_mask)[0] + 4
    n = 0
    while len(outliers) > 0:
        n +=1
        for i in outliers:
            
            if i < 15 or i > len(y)-15 :
                continue
            y[i] = (y[i -10 ] + y[i+10]) / 2
     #   plt.plot(x,y)
      #  plt.scatter(x[outliers],y[outliers],c='r')
        np.where(-1 == y)
        percentage_diff = np.abs(np.diff(y))
        rolling_avg = np.convolve(y, np.ones(5)/5, mode='valid',)
        diff_from_avg = (y[4:] - rolling_avg) /rolling_avg
        outliers_mask = diff_from_avg > 0.3
        # Replace outliers with the average of neighboring points
        outliers = np.where(outliers_mask)[0] + 4
        if n > 10:
            return data
    #plt.show()
    return y - max(data)

# =============================================================================
# def savefile(filepath,x,y):
#     with open(filepath, 'w') as file:
#         np.savetxt(file, np.column_stack((x,y)), delimiter=',')
#         file.write('\n')
# =============================================================================
        
def savefile(filepath, x, y):
    with open(filepath, 'w') as file:
        np.savetxt(file, np.column_stack((x, y)), delimiter=',')
        
#%%
# =============================================================================
# =============================================================================
# # Correction of 880 nm band, the 1060 dont need;
# =============================================================================
# =============================================================================
x_corr,y_corr = read_data(r'750B-Quantum Effieciency Suco.csv')
def smooth_data(numbers, window_size=10):
    smoothed_numbers = []

    for i in range(len(numbers)):
        start_index = max(0, i - window_size // 2)
        end_index = min(len(numbers), i + window_size // 2 + 1)

        # Extract the neighboring elements
        neighbors = numbers[start_index:end_index]

        # Calculate the average and append to the smoothed list
        smoothed_numbers.append(sum(neighbors) / len(neighbors))

    return smoothed_numbers

y_corr_func =  interpolate.interp1d(x_corr,y_corr,fill_value=0,bounds_error=False)

#%%
# =============================================================================
# 77 K
# =============================================================================
#path to folder with data:
correct = True
path = r'Corrected_data/EuDPA_60_77K_HQ_EmissionFilesForExcitationSpectra'
x, data = read_folder_ofSpectra_andPowerCorrect(path, num_columns=2, sep=',')
plt.plot(x, data[20])
#If background data provided:
Background_data = True
bkgpath = r'Corrected_data/BKG_77K__500ms_HQ_EmissionFilesForExcitationSpectra'
x, bkgdata = read_folder_ofSpectra_andPowerCorrect(bkgpath, num_columns=2, sep=',')

#Specify wavelength region 
wl = np.linspace(458,475,len(data))

#What is the range for Background subtraction:
condition1 = ((565 < x) & (x < 575))
#condition1 = ((560 < x) & (x < 572))  
condition2 = ((725 < x) & (x < 737))
#Was 733 - 737

# =============================================================================
# condition1 = ((565 < x) & (x < 575))
# condition2 = ((632.5 < x) & (x < 640))  
# condition3 = ((725 < x) & (x < 737))
# 
# =============================================================================
#Area to integrate of the emission peak
condition = [i for i, value in enumerate(x) if 575 < value and value < 725]

#Wavelengths ranges on the excitation side:
Wavelengths_ranges = [
    (wl > 464) & (474 > wl) ,
    ]

Focus_band =0

name = '300K_c880'
#%%
# =============================================================================
# Data cleaning
# =============================================================================
#Crappy backround:
#%matplotlib tk
data_corrected = []

plt.figure(dpi=300)
for i,dat in enumerate(data):     
    #determining bkg funtion
    x1,x2 = np.mean(x[condition1]),np.mean(x[condition2])
    y1,y2 = np.mean(dat[condition1]),np.mean(dat[condition2])
    bkgy = line_function(x1,y1,x2,y2, x)
    y_bkg_subtracted = dat - bkgy
    if Background_data:
        y1,y2 = np.mean(bkgdata[i][condition1]),np.mean(bkgdata[i][condition2])
        bkgy = line_function(x1,y1,x2,y2, x)
        bkgy_bkg_subtracted = bkgdata[i] - bkgy
        y_bkg_subtracted = y_bkg_subtracted - bkgy_bkg_subtracted
    #subtracting bkg:    
    y_bkg_subtracted = smoothing(x,y_bkg_subtracted)
    x1,x2 = np.mean(x[condition1]),np.mean(x[condition2])
    y1,y2 = np.mean(y_bkg_subtracted[condition1]),np.mean(y_bkg_subtracted[condition2])
    bkgy = line_function(x1,y1,x2,y2, x)
    y_corr = y_bkg_subtracted - bkgy
    plt.plot(x,y_corr)
    data_corrected.append(y_corr)

data_corrected = np.array(data_corrected)
emission_data = data_corrected

# =============================================================================
# for i, y in enumerate(emission_data):
#     path = '/Users/magnuswied/Desktop/Documents/SuCo/77K/Corrected_data'
#     filename = 'EuDPA_70_77k{}.csv'.format(round(wl[i], 1))
#     filepath = os.path.join(path, filename)
#     savefile(filepath, x, y)
# =============================================================================

print(data_corrected[0])
#%%
# =============================================================================
# Integrate for area and excitation plots from individual wavelengths in emission.
# ====================================================================


# =============================================================================
# # Read the corrected data from a new folder
# path = r'/Users/magnuswied/Desktop/Documents/SuCo/77K/Corrected_data'
# x, data_corr = read_file(path, num_columns=2, sep=',')
# 
# =============================================================================

x = []
corr_df = []

file_path_glob = glob.glob('Corrected_data/*.csv')

for i in file_path_glob:
    run = 0
    df = pd.read_csv(i, header=None, sep=',')
    corr_df.append(df.iloc[:,1])
    x.append(df.iloc[:,0])


data_corrected = np.array(corr_df)

wl = np.linspace(458,475,len(corr_df))
Wavelengths_ranges = [
    (wl > 464.0) & (474 > wl) ,
    ]

x = x[0]

#%%

print(len(corr_df))
#%%
#%matplotlib tk
area = []
area_tot = []
#condition_wl = [i for i, value in enumerate(wl) if 570 < value and value < 605]
condition = [i for i, value in enumerate(x) if 575 < value and value < 725]

for y in data_corrected:
    area.append(np.array(y[condition]).T)
    #Integrate peak with respect to condition:
    area_tot.append(-np.trapz(np.array(y[condition]),10**7/(np.array(x[condition]))))
    
area_tot = np.array(area_tot)
plt.plot(x,sum(data_corrected)/max(sum(data_corrected)),c = 'k',lw = 2)

print(len(area_tot))
#%%
# =============================================================================
# Excitation Spectrum:
# =============================================================================
Focus_band = 0
%matplotlib tk
fig, ax = plt.subplots(figsize=[8,6])
ax.plot(wl[Wavelengths_ranges[Focus_band]],area_tot[Wavelengths_ranges[Focus_band]],c = 'k',lw = 2)

len(Wavelengths_ranges[Focus_band])


ax.plot(wl,area_tot,c = 'k',lw = 2)
ax.set_xlabel('wavelength (nm)')
ax.set_ylabel('Intensity')
#ax.set_xlim(572,590)


#filepath = r'C:\Users\Villads\Documents\Project_NdpDO3A_Charlie\Optical data\DMSO\Generated data\300 K\Excitation{}.csv'.format(name)
#savefile(filepath,wl,area_tot)
#%%
# =============================================================================
# summed emission from each excitation band
# =============================================================================
Averaged_emission = []
for wl_r in Wavelengths_ranges:
    Averaged_emission.append(np.mean(data_corrected[wl_r],axis=0))
    

fig, ax = plt.subplots(figsize=[8,6])

plt.plot(x,Averaged_emission[0] / max(Averaged_emission[0]),label = 'band average {} nm'.format(round(np.mean(wl[Wavelengths_ranges[0]]))))

plt.legend()

ax.set_xlabel('Wavelength (nm)')
ax.set_ylabel('Normalized Intensity (a.u.)')
plt.title('Averaged Emission')
#filepath = r'C:\Users\Villads\Documents\Project_NdpDO3A_Charlie\Optical data\DMSO\Generated data\300 K\emission_average{}.csv'.format(name)
#savefile(filepath,x,Averaged_emission[1] / max(Averaged_emission[1]))
#%%
# =============================================================================
# Individual emission spectra Plotted
# =============================================================================
fig, ax = plt.subplots(figsize=[8,6])
cmap = plt.get_cmap('viridis')
emission_data = data_corrected[Wavelengths_ranges[Focus_band]]
colors = [cmap(i / len(emission_data)) for i in range(len(emission_data))]
lines = []
for i,y in enumerate(emission_data):
    ax.plot(x, y, color=colors[i])

plt.plot(x,Averaged_emission[Focus_band],label = 'Averaged', c = 'k', lw = 2.5)
plt.legend()

high = int(wl[Wavelengths_ranges[Focus_band]][-1])
low = int(wl[Wavelengths_ranges[Focus_band]][0])
sm = ScalarMappable(cmap=cmap, norm=plt.Normalize(low,high))
sm.set_array([])

cbar = plt.colorbar(sm, ticks=range(low,high, 5))
# Add the color bar legend
cbar.set_label('Excitation Wavelength (nm)')
ax.set_xlabel('Wavelength (nm)')
ax.set_ylabel('Intensity (a.u.)')
#%%
fig, ax = plt.subplots(figsize=[8,6])
cmap = plt.get_cmap('viridis')
emission_data = data_corrected[Wavelengths_ranges[Focus_band]]
colors = [cmap(i / len(emission_data)) for i in range(len(emission_data))]
lines = []
for i,y in enumerate(emission_data):
    ax.plot(x, y/max(y), color=colors[i])

plt.plot(x,Averaged_emission[Focus_band] / max(Averaged_emission[Focus_band]),label = 'Averaged', c = 'k', lw = 2.5)
plt.legend()

high = int(wl[Wavelengths_ranges[Focus_band]][-1])
low = int(wl[Wavelengths_ranges[Focus_band]][0])
sm = ScalarMappable(cmap=cmap, norm=plt.Normalize(low,high))
sm.set_array([])

cbar = plt.colorbar(sm, ticks=range(low,high, 5))
# Add the color bar legend
cbar.set_label('Excitation Wavelength (nm)')
ax.set_xlabel('Wavelength (nm)')
ax.set_ylabel('Normalized Intensity (a.u.)')

#%%
# =============================================================================
# Prepare for PCA
# =============================================================================
fig, ax = plt.subplots(figsize=[8,6])

wl_r = Wavelengths_ranges[Focus_band]
ax.plot(wl[wl_r],area_tot[wl_r],c = 'k')
ax.set_xlabel('Wavelength (nm)')
ax.set_ylabel('Intensity (a.u.)')
plt.title('Excitation range for PCA')

#%%

for i in data_corrected[wl_r]:
    print(len(i))
print(len(data_corrected[wl_r][0]))
#%%
Y = []
x_pca =x[condition]
fig, ax = plt.subplots(figsize=[8,6])
for y in data_corrected[wl_r]:
    y = y[condition]
    new_y = [(a - np.mean(y))/np.std(y) for a in y ]    
  #  new_y = y/max(y)
    plt.plot(x_pca,new_y)
    Y.append(new_y)
    # Print the number of True values for each y
ax.set_xlabel('Wavelength (nm)')
ax.set_ylabel('Intensity (a.u.)')
plt.title('input data for PCA')

#%%

print(len(Y))

#%%
# =============================================================================
# PCA Analysis
# =============================================================================


pca = PCA(n_components=3)
    
X_proj = pca.fit_transform(np.array(Y)) 
np.shape(X_proj)

cmap = plt.get_cmap('plasma')
colors = [cmap(i) for i in np.linspace(0, 1, len(X_proj))]
colors = [cmap(i) for i in (X_proj[:,1]-min(X_proj[:,1]))/max(X_proj[:,1]-min(X_proj[:,1]))]

#colors = [cmap(i) for i in np.linspace(0,1, len(yheat))]


pc_a,pc_b,pc_c =0,1,2
# Create the scatter plot
fig, ax = plt.subplots(figsize=[12, 6])
ax.scatter(X_proj[:, pc_a], X_proj[:, pc_b], color=colors, alpha=0.8)

# Set labels for the axes
ax.set_xlabel('PC{} ({}%)'.format(pc_a + 1, round(pca.explained_variance_ratio_[pc_a], 3)))
ax.set_ylabel('PC{} ({}%)'.format(pc_b + 1, round(pca.explained_variance_ratio_[pc_b], 3)))

# Define the font size for the labels
label_fontsize = 12  # Adjust the font size as needed

# Add integer labels for each data point with a specified font size
# =============================================================================
# for i, (a, b, c) in enumerate(zip(X_proj[:, pc_a], X_proj[:, pc_b],X_proj[:, pc_c])):
#     ax.annotate(str(i), (a, b), textcoords="offset points", xytext=(0, 10), ha='center', fontsize=label_fontsize)
# plt.title('PCA model')
# =============================================================================
plt.show()

individual_components = pca.components_ 

fig, ax = plt.subplots(figsize=[8,6])
plt.plot(x_pca,individual_components[0], label = 'PC1')
plt.plot(x_pca,individual_components[1], label = 'PC2')
ax.set_xlabel('Wavelength (nm)')
ax.set_ylabel('Intensity (a.u.)')
plt.legend()
plt.title('PCA components')

#%%
pca = PCA(n_components=3)
X_proj = pca.fit_transform(np.array(Y))

print(len(X_proj[0]))
#%%
# Define the PC indices
pc_a, pc_b, pc_c = 0, 1, 2

fig = plt.figure(figsize=[12, 6])
ax = fig.add_subplot(111, projection='3d')

# Create an array of indices
indices = np.arange(len(X_proj))

sc = ax.scatter(X_proj[:, pc_a], X_proj[:, pc_c], X_proj[:, pc_b], c=indices, cmap='plasma', alpha=0.8)

# Add color bar
cbar = fig.colorbar(sc, ticks=np.arange(len(X_proj)), format='%d')
cbar.set_label('Index')

# Set labels for the axes
ax.set_xlabel('PC{} ({}%)'.format(pc_a + 1, round(pca.explained_variance_ratio_[pc_a] * 100, 2)))
ax.set_zlabel('PC{} ({}%)'.format(pc_b + 1, round(pca.explained_variance_ratio_[pc_b] * 100, 2)))
ax.set_ylabel('PC{} ({}%)'.format(pc_c + 1, round(pca.explained_variance_ratio_[pc_c] * 100, 2)))

# Add integer labels for each data point with a specified font size
# =============================================================================
# for i, (a, b, c) in enumerate(zip(X_proj[:, pc_a], X_proj[:, pc_b], X_proj[:, pc_c])):
#     ax.text(a, b, c, str(i), fontsize=label_fontsize)
# 
# =============================================================================
plt.title('PCA model (3D)')
plt.show()

# Plot the individual PCA components
individual_components = pca.components_

fig, ax = plt.subplots(figsize=[8, 6])
ax.plot(x_pca, individual_components[0], label='PC1')
ax.plot(x_pca, individual_components[1], label='PC2')
ax.plot(x_pca, individual_components[2], label='PC3')  # Adding PC3 plot
ax.set_xlabel('Wavelength (nm)')
ax.set_ylabel('Intensity (a.u.)')
plt.legend()
plt.title('PCA components')
plt.show()

print(len(X_proj[:,pc_a]))
#%%

int_x = range(len(X_proj[:,pc_a]))
#print(len(int_x))
plt.plot(X_proj[:,pc_a], X_proj[:,pc_b], 'ro', markersize=3)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()

plt.plot(X_proj[:,pc_a], X_proj[:,pc_c], 'ro', markersize=3)
plt.xlabel('PC1')
plt.ylabel('PC3')
plt.show()

plt.plot(X_proj[:,pc_b], X_proj[:,pc_c], 'ro', markersize=3)
plt.xlabel('PC2')
plt.ylabel('PC3')
plt.show()

#%%
# =============================================================================
# 3D plot animation
# =============================================================================

pca = PCA(n_components=3)
X_proj = pca.fit_transform(np.array(Y))

# Define the PC indices
pc_a, pc_b, pc_c = 0, 1, 2

# Create the figure and axis
fig = plt.figure(figsize=[12, 6])
ax = fig.add_subplot(111, projection='3d')

# Set labels for the axes
ax.set_xlabel('PC{} ({}%)'.format(pc_a + 1, round(pca.explained_variance_ratio_[pc_a] * 100, 2)))
ax.set_ylabel('PC{} ({}%)'.format(pc_b + 1, round(pca.explained_variance_ratio_[pc_b] * 100, 2)))
ax.set_zlabel('PC{} ({}%)'.format(pc_c + 1, round(pca.explained_variance_ratio_[pc_c] * 100, 2)))

# Define the font size for the labels
label_fontsize = 12  # Adjust the font size as needed

# Set up the scatter plot
sc = ax.scatter([], [], [], color='b', alpha=0.8)

# Set limits for the axes
ax.set_xlim(np.min(X_proj[:, pc_a]), np.max(X_proj[:, pc_a]))
ax.set_ylim(np.min(X_proj[:, pc_b]), np.max(X_proj[:, pc_b]))
ax.set_zlim(np.min(X_proj[:, pc_c]), np.max(X_proj[:, pc_c]))

# Initialization function: plot the background of each frame
def init():
    sc._offsets3d = ([], [], [])
    return sc,

# Animation function: this is called sequentially
def animate(i):
    if i < len(X_proj):
        sc._offsets3d = ([X_proj[i, pc_a]], [X_proj[i, pc_b]], [X_proj[i, pc_c]])
    return sc,

# Call the animator
ani = FuncAnimation(fig, animate, frames=len(X_proj), init_func=init, interval=100, blit=True)

plt.title('PCA model (3D)')
plt.show()
#%%
# =============================================================================
# Reconstruction of spectra, Manual fingerprinting:
# =============================================================================
%matplotlib tk
X_reconstructed = pca.inverse_transform(X_proj)
plt.figure(figsize=(9, 6))
for i in [0]:  # Plot the first 5 spectra for illustration
    plt.plot(x_pca,Y[i], label='Original {}'.format(i), linewidth=2)
    plt.plot(x_pca,X_reconstructed[i], label='reconstructed = {}'.format(i))
    
    plt.legend()
   # plt.show()
plt.title('Spectral Reconstruction of 9 and 48')
plt.xlabel('Wavelengths (nm)')
plt.ylabel('Intensity (a.u.)')
ax.set_xlim(840,930)

#%%

X_score = pca.score_samples(Y)


print(X_proj)

plt.hist(X_score, bins=40)

#%%
func1 = X_reconstructed[22]
func2 = X_reconstructed[19] 

#%%
# =============================================================================
# Generation of fingerprint structures1
# =============================================================================
#Specify two spectra that well represents the most different ones. Then we subtract them from each other.
Spectra1 = 15
Spectra2 = 18
#Specify the scale used to subtract spectra 2 from 1 with
scale2 = 0.0

bkgrange1, bkgrange2 = ((560 < x_pca) & (x_pca < 572)) ,((733 < x_pca) & (x_pca < 737)) 

#bkg subtracts the reconstructed spectra, as they are generated with the mean set to 0, not the mininum.
bkg1 = line_function(np.mean(x_pca[bkgrange1]),np.mean(X_reconstructed[Spectra1][bkgrange1]),np.mean(x_pca[bkgrange2]),np.mean(X_reconstructed[Spectra1][bkgrange2]), x_pca)
recon_1 = X_reconstructed[Spectra1] - bkg1
bkg2 = line_function(np.mean(x_pca[bkgrange1]),np.mean(X_reconstructed[Spectra2][bkgrange1]),np.mean(x_pca[bkgrange2]),np.mean(X_reconstructed[Spectra2][bkgrange2]), x_pca)
recon_2 = (X_reconstructed[Spectra2] - bkg2) *scale2

#Plots the reconstructucted spectra along with the first species we are manually generating
%matplotlib inline

plt.figure(figsize=(9, 6))
plt.plot(x_pca,recon_1, label='recon1', linewidth=2)
plt.plot(x_pca,recon_2, label='scaled recon2', linewidth=2)
#%
species1 = recon_1 - recon_2
plt.plot(x_pca,species1 , label='species1 = recon1 - scaled recon2', linewidth=2)

plt.xlabel('Wavelengths (nm)')
plt.ylabel('Normalized Intensity (a.u.)')
plt.title('Manual construction of species1')
plt.legend()
#%%
# =============================================================================
# Generation of fingerprint structures
# =============================================================================
#Specify the scale used to subtract species 1 from spectra 2 with
scale2 = 0.2

#Plots the reconstructucted spectra along with the second species we are manually generating
plt.figure(figsize=(9, 6))
plt.plot(x_pca,species1 * scale2 , label='scaled species1', linewidth=2)
plt.plot(x_pca,recon_2 , label='recon2', linewidth=2)

species2 = recon_2*1 - species1 * scale2

plt.plot(x_pca,species2 , label='species2 = recon2 - scaled species1', linewidth=2)

plt.xlabel('Wavelengths (nm)')
plt.ylabel('Normalized Intensity (a.u.)')
plt.title('Manual construction of species2')
plt.legend()
#%%
plt.figure(figsize=(9, 6))
plt.plot(x_pca,species1/max(species1),label = 'Spectra A')
plt.plot(x_pca,species2/max(species2),label = 'Spectra B')



plt.legend()
plt.xlabel('Wavelengths (nm)')
plt.ylabel('Intensity (a.u.)')
plt.title('Manual Reconstructed Components')
#%%
def Fit_LinearComp(x,y,func1,func2):     
    Guesses = np.array([0.6,0.2,0])

    def Loss(arguments):
        a= arguments[0]
        b= arguments[1]
        c =arguments[2]
        return np.sum((func1 * a + func2 * b + c - y )**2)
    

    # Define constraints to ensure a and b are non-negative
    bounds = [(0, 10000000000)] + [(0, 10000000000)]  + [(0, 10000000000)]

    fit_values = minimize(Loss, Guesses, method='Powel''l', bounds=bounds).x
    a,b,c = fit_values[0],fit_values[1],fit_values[2]
    return a,b,c

A,B,CRAP = [],[],[]

func1 = extreme_point_positive_original[0] - min(extreme_point_positive_original[0] )
func2 = extreme_point_negative_original[0]- min(extreme_point_negative_original[0] )




x_fit = x[condition]
total_area = []
error =[]
for i in range(len(data_corrected)):

    y = data_corrected[i][condition]
    
    len(func1)
    
    a,b,c = Fit_LinearComp(x_fit,y,func1,func2)

# =============================================================================
#     A.append(a)
#     B.append(b)
# =============================================================================
    
    componentA = func1 * a
    A.append(-np.trapz(np.array(componentA),10**7/(np.array(x_fit))))
    componentB = func2 * b
    B.append(-np.trapz(np.array(componentB),10**7/(np.array(x_fit))))
    
    total_area.append(-np.trapz(np.array(y),10**7/(np.array(x_fit))))
    error.append(np.sum((func1 * a + func2 * b + c - y )**2  ))

    
    if i > 0:
        plt.plot(x_fit,func1 * a + func2 * b +c ,label = 'fit')
        plt.plot(x_fit,y,label = 'data')
        plt.title('Excitaiton Wavelength = {}'.format(wl[i]))
        plt.xlabel('wavelengths (nm)')
        plt.ylabel('Intensity (a.u.)')
        plt.legend()
        plt.show()

A= np.array(A)
B= np.array(B)
sum(A)/sum(B)

#%%
fig, (ax, ax2) = plt.subplots(2, 1, sharex=True, figsize=(8, 8))
ax.plot(wl,A, alpha = 0.5,label = 'component A')
ax.plot(wl,B, alpha = 0.5,label = 'component B')
ax.plot(wl,A+B,label = 'fit from PCA',c = 'r')

ax.plot(wl,total_area,c = 'k',label = 'measured excitation')
ax.legend()
ax2.plot(wl,np.array(np.sqrt(error)), label='data - fit_y')

ax2.legend()

ax2.set_xlabel('wavelengths (nm)')
ax.set_ylabel('Intensity (a.u.)')
ax2.set_ylabel('Error (a.u.)')

c:\Users\gusta\anaconda3\python.exe
['c:\\Users\\gusta\\Documents\\GitHub\\AMAS\\Project', 'C:\\Program Files\\PerkinElmerInformatics\\ChemOffice2019\\ChemScript\\Lib', 'c:\\Users\\gusta\\anaconda3\\python38.zip', 'c:\\Users\\gusta\\anaconda3\\DLLs', 'c:\\Users\\gusta\\anaconda3\\lib', 'c:\\Users\\gusta\\anaconda3', '', 'c:\\Users\\gusta\\anaconda3\\lib\\site-packages', 'c:\\Users\\gusta\\anaconda3\\lib\\site-packages\\win32', 'c:\\Users\\gusta\\anaconda3\\lib\\site-packages\\win32\\lib', 'c:\\Users\\gusta\\anaconda3\\lib\\site-packages\\Pythonwin']


FileNotFoundError: [WinError 3] Den angivne sti blev ikke fundet: 'Corrected_data/EuDPA_60_77K_HQ_EmissionFilesForExcitationSpectra'

In [ ]:
# =============================================================================
# =============================================================================
# # Second range fit:
# =============================================================================
# =============================================================================
path = r'C:\Users\Villads\Documents\Project_NdpDO3A_Charlie\Optical data\DMSO\300K\1070em\NdPDO3A_580_DMSO_300K_10s_1070_EmissionFilesForExcitationSpectra'
x, data = read_folder_ofSpectra_andPowerCorrect(path, num_columns=2, sep=',')
wl2 = np.linspace(566,607,len(data))

data_corrected2 = []

for i,dat in enumerate(data):     
    #determining bkg funtion
    x1,x2 = np.mean(x[condition1]),np.mean(x[condition2])
    y1,y2 = np.mean(dat[condition1]),np.mean(dat[condition2])
    
    bkgy = line_function(x1,y1,x2,y2, x)
    y_bkg_subtracted = dat - bkgy
    if Background_data:
        y1,y2 = np.mean(bkgdata[i][condition1]),np.mean(bkgdata[i][condition2])
        bkgy = line_function(x1,y1,x2,y2, x)
        bkgy_bkg_subtracted = bkgdata[i] - bkgy
        y_bkg_subtracted = y_bkg_subtracted - bkgy_bkg_subtracted


    #subtracting bkg:    
    y_bkg_subtracted = smoothing(x,y_bkg_subtracted)
    
    x1,x2 = np.mean(x[condition1]),np.mean(x[condition2])
    y1,y2 = np.mean(y_bkg_subtracted[condition1]),np.mean(y_bkg_subtracted[condition2])
    bkgy = line_function(x1,y1,x2,y2, x)
    y_corr = y_bkg_subtracted - bkgy
   # plt.plot(x,y_corr)

    data_corrected2.append(smoothing(x,y_corr))
data_corrected2 = np.array(data_corrected2)

In [ ]:
def Fit_LinearComp(x,y,func1,func2):     
    Guesses = np.array([0.6,0.2,0])

    def Loss(arguments):
        a= arguments[0]
        b= arguments[1]
        c =arguments[2]
        return np.sum((func1 * a + func2 * b + c - y )**2)
    

    # Define constraints to ensure a and b are non-negative
    bounds = [(0, 10000000000)] + [(0, 10000000000)]  + [(0, 10000000000)]

    fit_values = minimize(Loss, Guesses, method='Powel''l', bounds=bounds).x
    a,b,c = fit_values[0],fit_values[1],fit_values[2]
    return a,b,c

A,B,CRAP = [],[],[]

func1 = extreme_point_positive_original[0] - min(extreme_point_positive_original[0] )
func2 = extreme_point_negative_original[0]- min(extreme_point_negative_original[0] )

x_fit = x[condition]
total_area = []
error =[]
for i in range(len(data_corrected2)):

    y = data_corrected2[i][condition]
    
    len(func1)
    
    a,b,c = Fit_LinearComp(x_fit,y,func1,func2)

In [ ]:
# =============================================================================
#     A.append(a)
#     B.append(b)
# =============================================================================
    
    componentA = func1 * a
    A.append(-np.trapz(np.array(componentA),10**7/(np.array(x_fit))))
    componentB = func2 * b
    B.append(-np.trapz(np.array(componentB),10**7/(np.array(x_fit))))
    
    total_area.append(-np.trapz(np.array(y),10**7/(np.array(x_fit))))
    error.append(np.sum((func1 * a + func2 * b + c - y )**2  ))

# =============================================================================
#     if (i  < 220 and i > 210):
#         plt.plot(x_fit,func1 * a + func2 * b +c ,label = 'fit')
#         plt.plot(x_fit,y,label = 'data')
#         plt.title('Excitaiton Wavelength = {}'.format(wl[i]))
#         plt.xlabel('wavelengths (nm)')
#         plt.ylabel('Intensity (a.u.)')
#         plt.legend()
#         plt.show()
# =============================================================================

A= np.array(A)
B= np.array(B)
#%%
fig, (ax, ax2) = plt.subplots(2, 1, sharex=True, figsize=(8, 8))
ax.plot(wl2,A, alpha = 0.5,label = 'component A')
ax.plot(wl2,B, alpha = 0.5,label = 'component B')
ax.plot(wl2,A+B,label = 'fit from PCA',c = 'r')

ax.plot(wl2,total_area,c = 'k',label = 'measured excitation')
ax.legend()
ax2.plot(wl2,np.array(np.sqrt(error)), label='data - fit_y')

ax2.legend()

ax2.set_xlabel('wavelengths (nm)')
ax.set_ylabel('Intensity (a.u.)')
ax2.set_ylabel('Error (a.u.)')